# Building Neural Networks

Neural networks are constructed with `torch.nn`.

A `nn.Module` will have layers, and a `forward(input)` method that returns the `output`.

Let's define a network. *Warning: We really start to speed up here*. 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


We will feed a random 32x32 image. The default input size is 32x32 (I don't feel this is clear from the code above and I cannot really figure out why). Apparently this is LeNet as well, I also don't feel like this is clear. 

In [24]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1174, -0.0246, -0.1157,  0.0345, -0.0008, -0.0228,  0.0266,  0.1039,
          0.1126,  0.0755]], grad_fn=<AddmmBackward0>)


Zero the gradient buffers of all parameters and backprops with random gradients. 

In [25]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## Loss Function

In [26]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.4119, grad_fn=<MseLossBackward0>)


In [27]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [28]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0005, -0.0030, -0.0031, -0.0045, -0.0012, -0.0082])


## Update the Weights

For stochastic gradient descent:
`weight = weight - learning rate * gradi